In [ ]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/Landslide_dataSet1.csv"
#device = "102"
#save_path = f"./data/dev{device}_prepared.csv"
save_path = f"data/dataSet1_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

df = df.drop(columns=['unixt', 'gateway time', 'old time', 'risk_level', 'confidence'], errors='ignore')
df = df[df['devID'] != 218499049.0]
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] != 0.0]
df = df[df['rain'] != -1.0]

#df = df[df['devID'] == int(device)]
df

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
0,2025-07-30 03:28:43.522,101,24.400000,0.0,35.753414,54.635536,17.443285,8.638051,99.898964
1,2025-07-30 03:28:44.756,102,20.900000,0.0,36.623943,51.436863,2.512411,8.637977,99.898933
2,2025-07-30 03:28:45.992,101,24.500000,0.0,35.700008,54.654610,11.029456,8.638051,99.898964
3,2025-07-30 03:28:47.259,101,24.400000,0.0,35.673306,55.825741,8.358195,8.638053,99.898964
4,2025-07-30 03:28:48.473,102,20.900000,0.0,36.591900,53.292744,2.527225,8.637976,99.898933
...,...,...,...,...,...,...,...,...,...
34176,2025-08-03 07:43:15.179,101,26.299999,0.0,34.415577,70.317993,2.514839,8.638032,99.898972
34177,2025-08-03 07:43:24.004,102,23.400000,0.0,35.267414,69.381470,3.813702,8.637955,99.898911
34178,2025-08-03 07:43:26.139,101,26.299999,0.0,34.388874,70.276031,10.165874,8.638032,99.898972
34179,2025-08-03 07:43:35.249,102,23.400000,0.0,35.286106,65.318764,3.002546,8.637956,99.898911


In [2]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
count,34160,34160.000000,34160.000000,34160.000000,34160.000000,34160.000000,34160.000000,34160.000000,34160.000000
mean,2025-08-01 22:12:57.081876736,101.502196,22.503047,0.095467,30.529776,74.525361,9.061593,8.637981,99.898949
min,2025-07-30 03:28:43.522000,101.000000,19.200001,0.000000,25.934618,46.090485,0.504865,8.637903,99.898880
25%,2025-08-01 12:57:12.711250176,101.000000,19.700001,0.000000,27.430000,66.769325,2.969754,8.637939,99.898926
50%,2025-08-02 03:20:38.166500096,102.000000,20.900000,0.000000,28.774511,79.780502,8.046375,8.637980,99.898949
75%,2025-08-02 17:34:59.387500032,102.000000,24.500000,0.000000,34.354828,81.537193,14.540179,8.638021,99.898972
max,2025-08-03 07:43:37.347000,102.000000,37.500000,15.367000,38.156708,85.678116,32.059963,8.638075,99.899017
std,NaN,0.500002,3.247636,1.028058,3.546256,9.716323,6.280863,0.000043,0.000026


In [3]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
0,2025-07-30 03:28:43.522,101,24.400000,0.0,35.753414,54.635536,17.443285,8.638051,99.898964
1,2025-07-30 03:28:44.756,102,20.900000,0.0,36.623943,51.436863,2.512411,8.637977,99.898933
2,2025-07-30 03:28:45.992,101,24.500000,0.0,35.700008,54.654610,11.029456,8.638051,99.898964
3,2025-07-30 03:28:47.259,101,24.400000,0.0,35.673306,55.825741,8.358195,8.638053,99.898964
4,2025-07-30 03:28:48.473,102,20.900000,0.0,36.591900,53.292744,2.527225,8.637976,99.898933
...,...,...,...,...,...,...,...,...,...
34176,2025-08-03 07:43:15.179,101,26.299999,0.0,34.415577,70.317993,2.514839,8.638032,99.898972
34177,2025-08-03 07:43:24.004,102,23.400000,0.0,35.267414,69.381470,3.813702,8.637955,99.898911
34178,2025-08-03 07:43:26.139,101,26.299999,0.0,34.388874,70.276031,10.165874,8.638032,99.898972
34179,2025-08-03 07:43:35.249,102,23.400000,0.0,35.286106,65.318764,3.002546,8.637956,99.898911


In [4]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek
df = df.reset_index() 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34160 entries, 0 to 34159
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    34160 non-null  datetime64[ns]
 1   devID        34160 non-null  int64         
 2   soil         34160 non-null  float64       
 3   rain         34160 non-null  float64       
 4   temp         34160 non-null  float64       
 5   humi         34160 non-null  float64       
 6   geo          34160 non-null  float64       
 7   lat          34160 non-null  float64       
 8   lng          34160 non-null  float64       
 9   hour         34160 non-null  int32         
 10  day_of_week  34160 non-null  int32         
dtypes: datetime64[ns](1), float64(7), int32(2), int64(1)
memory usage: 2.6 MB


In [6]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [7]:
df = df.dropna(subset=['lng']).reset_index(drop=True)

In [8]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [9]:
df = df.dropna()

nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("df102")
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

df102
📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [10]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
